In [ ]:
"""
把从上交所和深交所下载的年报pdf，提取文本

为了加速，采用多进程并发方式
"""

In [ ]:
#!jupyter nbconvert --to script pdf_to_txt.ipynb
import os

def getSize(txt_filename):
    fsize=0
    try:
        if os.path.exists(txt_filename):
            fsize = os.path.getsize(txt_filename)
            print(f'【进程{os.getpid()}】file {txt_filename} size is {int(fsize/1024)}kb')
    except Exception as e:
        print(f'【进程{os.getpid()}】getsize exception {txt_filename} ',e)
    return fsize

def pdf_to_txt(pdf_file,txt_dir='./'):
    import subprocess
    import os
    try:
        txt_filename=txt_dir+os.path.basename(pdf_file).replace('pdf','txt')
        fsize = getSize(txt_filename)
        if fsize > 1024*10:
            print(f'【进程{os.getpid()}】jump...')
            return 

        fsize = 0
        try:
            if os.path.exists(pdf_file):
                fsize = os.path.getsize(pdf_file)
        except Exception as e:
            return 
#         out_bytes = subprocess.check_output(['which','pdf2txt.py'])
#         pdf2txt=out_bytes.decode("utf-8").replace('\n','')
#         if not pdf2txt:
#             print('pdf2txt.py 不存在，程序退出<c')
#             quit()
#         subprocess.run(["python", pdf2txt, "-o",txt_filename ,pdf_file])
        t0 = time.time()
        print(f'【进程{os.getpid()}】开始转换文件：{pdf_file}')
        subprocess.run(["pdf2txt.py", "-o",txt_filename ,pdf_file])
        fsize_ret = getSize(txt_filename)
        print(f'【进程{os.getpid()}】转换文件成功：{pdf_file}，用时={int(time.time()-t0)}秒, 转换速度={int(fsize/1024/(time.time()-t0))}KB/秒,txt大小={int(fsize_ret/1024)}KB,pdf大小={int(fsize/1024)}KB')
        if fsize_ret < 1024:
            print(f'【进程{os.getpid()}】文件太小{fsize_ret}，删除原文件{pdf_file}')
            os.remove(txt_filename)
            os.remove(pdf_file)
            
    except subprocess.CalledProcessError as e:
        out_bytes = e.output       # Output generated before error
        code      = e.returncode   # Return code
        print("E1"*10,e)
    except Exception as e:
        print("E2"*10,e)
     

    


# pdf_file=pdf_dir+'000780_2006.pdf' 
# pdf_to_txt(pdf_file)

In [ ]:
# find files

def find_files(pdf_dir):
    return [pdf_dir+f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

pdf_dir ='../scraping/pdf/'
pdf_files = find_files(pdf_dir)
print(len(pdf_files))
print(pdf_files[:4])

In [ ]:


base_dir='../scraping/content/'
import time  
from multiprocessing import Process, Queue
from tqdm import tqdm

q = Queue()

def f(q: Queue):
    while True:
        pdf_file = q.get()
        pdf_to_txt(pdf_file,base_dir)



            
def loop_task():
    import time  
    for pdf_file in tqdm(pdf_files):
        if q.qsize()> 1000:
            print("sleep for queue is not empty")
            time.sleep(60)  # sleep 10秒
        else:
            q.put(pdf_file)


# 启动四个进程进行处理
# python  down_pdf.py 3 1 表示启动三个进程，除以3取摸为1的进行
import sys
process_num=2
print("sys.argv=",sys.argv)
if len(sys.argv)==2:
    process_num=int(sys.argv[1])
    
for i in range(process_num):
    p = Process(target=f, args=(q,))
    p.start()

loop_task()

    